In [1]:
!pip install openpyxl
!pip install imbalanced-learn
!pip3 install ipympl
!pip install import-ipynb
!pip install shapely
!pip install SciencePlots 
!pip install seaborn
!pip install tqdm
!pip install ipywidgets


[notice] A new release of pip is available: 23.2 -> 23.2.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.2 -> 23.2.1
[notice] To update, run: pip install --upgrade pip



[notice] A new release of pip is available: 23.2 -> 23.2.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.2 -> 23.2.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.2 -> 23.2.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.2 -> 23.2.1
[notice] To update, run: pip install --upgrade pip



[notice] A new release of pip is available: 23.2 -> 23.2.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.2 -> 23.2.1
[notice] To update, run: pip install --upgrade pip



[notice] A new release of pip is available: 23.2 -> 23.2.1
[notice] To update, run: pip install --upgrade pip


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scienceplots
import os, sys
from numpy import nan
import re
import ipympl
# from IPython.core.display import display, HTML
import ipywidgets
import json
from os import listdir
import glob
import math
from IPython.display import Image, display, HTML
from shapely.geometry import mapping
from sklearn.cross_decomposition import PLSRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score, train_test_split, RepeatedKFold, cross_val_predict
from sklearn.metrics import r2_score, mean_squared_error, precision_score, recall_score, mean_absolute_error, make_scorer
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression
from sklearn.kernel_ridge import KernelRidge
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, LeaveOneOut, cross_validate
from sklearn.impute import SimpleImputer
from tqdm.notebook import tqdm
from sklearn.svm import SVC
from tqdm import tqdm
import pickle
from imblearn.over_sampling import SMOTE
from scipy.stats import ks_2samp
from scipy import stats
np.seterr(divide='ignore', invalid='ignore')
pd.options.display.max_columns = 100
from sklearn.preprocessing import PolynomialFeatures
%matplotlib inline

In [3]:
import import_ipynb
from SoilPrep import * 

importing Jupyter notebook from SoilPrep.ipynb


# Step 1: Preprocessing (Normalisation and Smoothing)

In [4]:
# Colour scheme definition
kado = '#8B7355'
mati = '#A52A2A'
balu = '#F4A460'


df = pd.read_csv('uae.csv')

# ------------- Target Isolation ----------------------
target_names = ['sand', 'silt', 'clay', 'TOC']
clr = ['#F4A460', '#8B7355', '#A52A2A', 'green']
def isolate_targets(df, target_names):
    T=[]
    for i in range (0,len(target_names)):
        T.append(df[target_names[i]])
    return(T)
    
T = isolate_targets(df,target_names) 

# -------------- Spectra and Smoothed Spectra-----------
spectra = df.iloc[:, 7:2158].copy()
c_min =0
c_max =2150

smth_spec =[]
for i in range (0,51):
    smth_spec.append(sgsmooth(spectra, i))


In [5]:
# --- TODO : Also define and Use other normalisation methods ----
def normalize_targets(T):          
    NT =[]
    for i in range(0, len(T)):
        NT.append(lognormal(T[i].copy()))
    return(NT)

NT = normalize_targets(T)


## Correlation between wavelengths and Targets

In [6]:
plt.style.use(['science','notebook','grid'])
# Pearson corelation between different wavelengths and Targets/Outputs (i.e, sand, clay, silt, and TOC) 

def find_rpval (spectra, T):
    r_val = []
    p_val = []
    for i in range (0,len(T)):
        p_val.append(spectra.loc[[0],:].copy())
        r_val.append(spectra.loc[[0],:].copy())
        for j in range(c_min, c_max+1):
            r_val[i].iloc[0,j], p_val[i].iloc[0,j] = stats.pearsonr(T[i], spectra.iloc[:, j])
    return(r_val, p_val)

r_val, p_val = find_rpval (spectra, T)

In [12]:
light_blue = '#6495ED'
spec_std = 6*spectra.std()
mean_spec_std = spec_std.copy()
mean_spec_std.iloc[:] = spec_std.mean()

def plot_corr (target, mean_of_std):
    i = target_names.index(target)
    r_val[i].iloc[0,:].plot(color = clr[i])
    spec_std.plot(color = 'blue')
    if mean_of_std == 'on':
        mean_spec_std.plot(color = 'blue', linewidth=1, linestyle='dashed')
    plt.ylim([-0.6, 0.6])

ipywidgets.interact(plot_corr, target = target_names, mean_of_std= ['on', 'off'])

interactive(children=(Dropdown(description='target', options=('sand', 'silt', 'clay', 'TOC'), value='sand'), D…

<function __main__.plot_corr(target, mean_of_std)>

## Visualizing Smoothed Spectrum (variable window)

In [13]:
# ---- TODO: display the change in each target content (from previous) using little bars 
# ---- TODO: include continium removed spectra for better visualization ---
def plot_spec (sample, window):
    x = smth_spec[window].iloc[sample,:]
    x.plot()
    plt.ylim([-0.6, 0.8])

ipywidgets.interact(plot_spec, sample = (0, 293,1), window = (0,49,1))

interactive(children=(IntSlider(value=146, description='sample', max=293), IntSlider(value=24, description='wi…

<function __main__.plot_spec(sample, window)>

# Step 2:  Best Train-Test Split  (Parameters)

In [21]:
tst_siz = 0.2

rand_t = [None] * (len(T))
err_t = [None] * (len(T))

print('Without Normalization:')
for i in range (0,len(T)):
    rand_t[i], err_t[i] = best_split(spectra.copy(), T[i], tst_siz) 
    print ('For '+ target_names[i]+ ' :test size =', tst_siz, '\t min bin error=', err_t[i], '\t at randome state =', rand_t[i])
    
rand_nt = [None] * (len(T))
err_nt = [None] * (len(T))

print('After Normalization:')
for i in range (0,len(T)):
    rand_nt[i], err_nt[i] = best_split(spectra.copy(), NT[i], tst_siz)     
    print ('For '+ target_names[i]+ ' :test size =', tst_siz, '\t min bin error=', err_nt[i], '\t at randome state =', rand_nt[i])
    
        

Without Normalization:
For sand :test size = 0.2 	 min bin error= 66.25 	 at randome state = 21
For silt :test size = 0.2 	 min bin error= 48.75 	 at randome state = 2
For clay :test size = 0.2 	 min bin error= 33.75 	 at randome state = 7
For TOC :test size = 0.2 	 min bin error= 41.25 	 at randome state = 24
After Normalization:
For sand :test size = 0.2 	 min bin error= 78.75 	 at randome state = 10
For silt :test size = 0.2 	 min bin error= 53.75 	 at randome state = 23
For clay :test size = 0.2 	 min bin error= 33.75 	 at randome state = 7
For TOC :test size = 0.2 	 min bin error= 66.25 	 at randome state = 1



##  Visualizing the Optimal Test Train Split 

In [24]:
def best_split_plot (target):
    
    i = target_names.index(target)
    
    X_train, X_test, y_train, y_test = train_test_split(spectra, T[i], test_size= tst_siz, random_state=rand_t[i])
    NX_train, NX_test, Ny_train, Ny_test = train_test_split(spectra, NT[i], test_size= tst_siz, random_state=rand_nt[i])
    
    fig, ax = plt.subplots(1,2, figsize=(18,7))
    
    binsT = np.linspace(np.min(T[i]), np.max(T[i]),10)
    ax[0].hist([y_train, y_test], binsT , label=['Train', 'Test'], density=True, color = ['blue','red'])
    ax[0].legend(loc='upper center', fontsize =12)
    ax[0].set_xlabel(target_names[i]+' content',fontsize =16)
    ax[0].set_ylabel('Normalised frequency',fontsize =12)
    ax[0].tick_params(axis='both', labelsize=8)
    
    binsNT = np.linspace(np.min(NT[i]), np.max(NT[i]),10)
    ax[1].hist([Ny_train, Ny_test], binsNT , label=['Train', 'Test'], density=True, color = ['blue','red'])
    ax[1].legend(loc='upper center', fontsize =12)
    ax[1].set_xlabel('Normalised '+target_names[i]+' content',fontsize =16)
    ax[1].set_ylabel('Normalised frequency',fontsize =12)
    ax[1].tick_params(axis='both', labelsize=8)

    fig.suptitle('Train Test Distribution of Data', x = 0.5 ,y = .95, fontsize = 20)
    

ipywidgets.interact(best_split_plot, target = target_names)


interactive(children=(Dropdown(description='target', options=('sand', 'silt', 'clay', 'TOC'), value='sand'), O…

<function __main__.best_split_plot(target)>

# Step 3: Best Model Parameters (window, n_components)

## PLSR: obtaining best parameter (window, n_components, IQRP)

In [109]:
from sklearn import linear_model

def best_param_PLSR (spectra, target, rand_st, tst_siz, window, n_comp):
    X= spectra.copy()
    y= target.copy()
     
    iqrpM = np.zeros(shape=(window, n_comp))
    iqrpMtr = np.zeros(shape=(window, n_comp))
    
    for w in range (0,window):
        X_train, X_test, y_train, y_test = train_test_split(X[w], y, test_size = tst_siz, random_state=rand_st)
        
        for n in range(1,n_comp):
            Model = PLSRegression(n_components=n, scale=True)
            
            Model.fit(X_train, y_train)
            y_pred = Model.predict(X_test, copy=True)
            yhat_pred = Model.predict(X_train, copy=True)            
            
            iqrp_test = iqrp(y_pred, y_test)
            iqrp_train = iqrp(yhat_pred, y_train)
            
            iqrpM[w,n] = iqrp_test
            iqrpMtr[w,n] = iqrp_train
            
            if iqrp_test == iqrpM.max():
                bestw = w
                bestn = n
                maxIQRP = iqrp_test
                #print('Found new best at w=', w, 'n=', n,  '>>   IQRP_test =', iqrp_test, '>>   IQRP_train =', iqrp_train)
                
                
            if iqrp_train == iqrpMtr.max():
                bestwT = w
                bestnT = n
                maxIQRPT = iqrp_train
                
    #print('On train data best at w=', bestwT, 'n=', bestnT,  '>>   IQRP_train =', maxIQRPT)
                
                
    return (bestw, bestn, maxIQRP)


w = [None] * (len(T))
n_comp = [None] * (len(T))
IQRP = [None] * (len(T))

wN = [None] * (len(T))
n_compN = [None] * (len(T))
IQRPN = [None] * (len(T))

for i in range (0,len(T)):
    w[i], n_comp[i], IQRP[i] = best_param_PLSR (smth_spec.copy(), T[i], rand_t[i], 0.2, 5,30)
    wN[i], n_compN[i], IQRPN[i] = best_param_PLSR (smth_spec.copy(), NT[i], rand_nt[i], 0.2, 5,30)

In [110]:
# w1, n1, IQRP1 = best_param_PLSR (smth_spec.copy(), T[3], rand_t[3], 0.2, 5,30)
# w1, n1, IQRP1 = best_param_PLSR (smth_spec.copy(), NT[3], rand_nt[3], 0.2, 5,30)

##  Multilinear: obtaining best parameters(window and IQRP)

In [131]:
def best_param_Mult (spectra, target, rand_st, tst_siz, window):
    X= spectra.copy()
    y= target.copy()
     
    iqrpM = np.zeros(shape=(window, 1))
    iqrpMtr = np.zeros(shape=(window, 1))
    
    for w in range (0,window):
        X_train, X_test, y_train, y_test = train_test_split(X[w], y, test_size = tst_siz, random_state=rand_st)
        Model = linear_model.LinearRegression()
            
        Model.fit(X_train, y_train)
        y_pred = Model.predict(X_test)
        yhat_pred = Model.predict(X_train)            
            
        iqrp_test = iqrp(y_pred, y_test)
        iqrp_train = iqrp(yhat_pred, y_train)
            
        iqrpM[w,0] = iqrp_test
        iqrpMtr[w,0] = iqrp_train
        if iqrp_test == iqrpM.max():
                bestw = w
                #bestn = n
                maxIQRP = iqrp_test
                #print('Found new best at w=', w, '>>   IQRP_test =', iqrp_test, '>>   IQRP_train =', iqrp_train)
         
        if iqrp_train == iqrpMtr.max():
                bestwT = w
                #bestnT = n
                maxIQRPT = iqrp_train
        
        
        
    #print('On train data best at w=', bestwT, '>>   IQRP_train =', maxIQRPT)
                
                
    return (bestw, maxIQRP)

w_mult = [None] * (len(T))
IQRP_mult = [None] * (len(T))

wN_mult = [None] * (len(T))
IQRPN_mult = [None] * (len(T))

for i in range (0,len(T)):
    w_mult[i], IQRP_mult[i] = best_param_Mult (smth_spec.copy(), T[i], rand_t[i], 0.2, 5)
    wN_mult[i], IQRPN_mult[i] = best_param_Mult (smth_spec.copy(), NT[i], rand_nt[i], 0.2, 5)
    

In [134]:
#w1, IQRP1 = best_param_Mult (smth_spec.copy(), T[1], rand_t[1], 0.2, 5)
# w1,  IQRP1 = best_param_Mult (smth_spec.copy(), NT[1], rand_nt[1], 0.2, 45)

## Visualizing Model Accuracy (PLSR)

In [143]:
def plot_model_acc (target, method1, method2):
    
    i = target_names.index(target)  
#--------------------------------------------- setting variables for method1 plot----------------    
    if method1 == 'PLSR':
        m1_wT = w[i]
        m1_n_compT = n_comp[i]
        m1_IQRPT = IQRP[i]
    
        m1_wNT = wN[i]
        m1_n_compNT = n_compN[i]
        m1_IQRPNT = IQRPN[i]
    
        # performing optimum train test split (note that it needs wT due to smth_spec[wT])
        X1_train, X1_test, y1_train, y1_test = train_test_split(smth_spec[m1_wT], T[i], test_size= 0.2, random_state=rand_t[i])
        NX1_train, NX1_test, Ny1_train, Ny1_test = train_test_split(smth_spec[m1_wNT], NT[i], test_size= 0.2, random_state=rand_nt[i])

        # initializing regression model for the target and normalized target
        m1_Model = PLSRegression(n_components=m1_n_compT, scale=True)
        m1_ModelN = PLSRegression(n_components=m1_n_compNT, scale=True)
    
        # fitting PLSR model and prediction on test and normalized test data
        m1_Model.fit(X1_train, y1_train)
        m1_ModelN.fit(NX1_train, Ny1_train)
    
        y1_pred = m1_Model.predict(X1_test,copy=True)
        y1_tp = pd.DataFrame({'actual':y1_test.values, 'predic': y1_pred[:,0]})
        z1 = np.polyfit(y1_test, y1_pred, 1)
        score1_cv = r2_score(y1_test, y1_pred)
    
        Ny1_pred = m1_ModelN.predict(NX1_test,copy=True)        
        Ny1_tp = pd.DataFrame({'actual':Ny1_test.values, 'predic': Ny1_pred[:,0]})
        z1N = np.polyfit(Ny1_test, Ny1_pred, 1)
        score1_cvN = r2_score(Ny1_test, Ny1_pred)
        
    elif method1 == 'MULT':
        m1_wT = w_mult[i]
        
        m1_IQRPT = IQRP_mult[i]
    
        m1_wNT = wN_mult[i]
        
        m1_IQRPNT = IQRPN_mult[i]
    
        # performing optimum train test split (note that it needs wT due to smth_spec[wT])
        X1_train, X1_test, y1_train, y1_test = train_test_split(smth_spec[m1_wT], T[i], test_size= 0.2, random_state=rand_t[i])
        NX1_train, NX1_test, Ny1_train, Ny1_test = train_test_split(smth_spec[m1_wNT], NT[i], test_size= 0.2, random_state=rand_nt[i])

        # initializing regression model for the target and normalized target
        m1_Model = linear_model.LinearRegression()
        m1_ModelN = linear_model.LinearRegression()
    
        # fitting MULT model and prediction on test and normalized test data
        m1_Model.fit(X1_train, y1_train)
        m1_ModelN.fit(NX1_train, Ny1_train)
    
        y1_pred = m1_Model.predict(X1_test)
        y1_tp = pd.DataFrame({'actual':y1_test.values, 'predic': y1_pred.copy()})
        z1 = np.polyfit(y1_test, y1_pred, 1)
        score1_cv = r2_score(y1_test, y1_pred)
    
        Ny1_pred = m1_ModelN.predict(NX1_test)        
        Ny1_tp = pd.DataFrame({'actual':Ny1_test.values, 'predic': Ny1_pred.copy()})
        z1N = np.polyfit(Ny1_test, Ny1_pred, 1)
        score1_cvN = r2_score(Ny1_test, Ny1_pred)   
    

#--------------------------------------------- setting variables for method2 plot----------------     
    
    if method2 == 'PLSR':
        m2_wT = w[i]
        m2_n_compT = n_comp[i]
        m2_IQRPT = IQRP[i]
    
        m2_wNT = wN[i]
        m2_n_compNT = n_compN[i]
        m2_IQRPNT = IQRPN[i]
    
        # performing optimum train test split (note that it needs wT due to smth_spec[wT])
        X2_train, X2_test, y2_train, y2_test = train_test_split(smth_spec[m2_wT], T[i], test_size= 0.2, random_state=rand_t[i])
        NX2_train, NX2_test, Ny2_train, Ny2_test = train_test_split(smth_spec[m2_wNT], NT[i], test_size= 0.2, random_state=rand_nt[i])

        # initializing regression model for the target and normalized target
        m2_Model = PLSRegression(n_components=m2_n_compT, scale=True)
        m2_ModelN = PLSRegression(n_components=m2_n_compNT, scale=True)
    
        # fitting PLSR model and prediction on test and normalized test data
        m2_Model.fit(X2_train, y2_train)
        m2_ModelN.fit(NX2_train, Ny2_train)
    
        y2_pred = m2_Model.predict(X2_test,copy=True)
        y2_tp = pd.DataFrame({'actual':y2_test.values, 'predic': y2_pred[:,0]})
        z2 = np.polyfit(y2_test, y2_pred, 1)
        score2_cv = r2_score(y2_test, y2_pred)
    
        Ny2_pred = m2_ModelN.predict(NX2_test,copy=True)        
        Ny2_tp = pd.DataFrame({'actual':Ny2_test.values, 'predic': Ny2_pred[:,0]})
        z2N = np.polyfit(Ny2_test, Ny2_pred, 1)
        score2_cvN = r2_score(Ny2_test, Ny2_pred)
        
    elif method2 == 'MULT':
        m2_wT = w_mult[i]
        
        m2_IQRPT = IQRP_mult[i]
    
        m2_wNT = wN_mult[i]
        
        m2_IQRPNT = IQRPN_mult[i]
    
        # performing optimum train test split (note that it needs wT due to smth_spec[wT])
        X2_train, X2_test, y2_train, y2_test = train_test_split(smth_spec[m2_wT], T[i], test_size= 0.2, random_state=rand_t[i])
        NX2_train, NX2_test, Ny2_train, Ny2_test = train_test_split(smth_spec[m2_wNT], NT[i], test_size= 0.2, random_state=rand_nt[i])

        # initializing regression model for the target and normalized target
        m2_Model = linear_model.LinearRegression()
        m2_ModelN = linear_model.LinearRegression()
    
        # fitting MULT model and prediction on test and normalized test data
        m2_Model.fit(X2_train, y2_train)
        m2_ModelN.fit(NX2_train, Ny2_train)
    
        y2_pred = m2_Model.predict(X2_test)
        y2_tp = pd.DataFrame({'actual':y2_test.values, 'predic': y2_pred.copy()})
        z2 = np.polyfit(y2_test, y2_pred, 1)
        score2_cv = r2_score(y2_test, y2_pred)
    
        Ny2_pred = m2_ModelN.predict(NX2_test)        
        Ny2_tp = pd.DataFrame({'actual':Ny2_test.values, 'predic': Ny2_pred.copy()})
        z2N = np.polyfit(Ny2_test, Ny2_pred, 1)
        score2_cvN = r2_score(Ny2_test, Ny2_pred)     
    
    
    fig, axes = plt.subplots(2,2, figsize=(16,16))
    
    #with plt.style.context(('ggplot')):  ------- PLOT for Model 1--------------------------------------
    y1_tp.plot.scatter(ax= axes[0][0], x="actual", y="predic", alpha=0.8, color = clr[i], edgecolors='k')
    axes[0][0].plot(y1_test, np.polyval(z1, y1_test),  c='blue', linewidth=1)
    axes[0][0].plot(y1_test, y1_test, color='green', linewidth=1)
    axes[0][0].tick_params(axis='both', labelsize=10)
    axes[0][0].text(0.05, 0.95, target_names[i], transform=axes[0][0].transAxes, fontsize = 20, color = clr[i])
    axes[0][0].text(0.05, 0.90, 'IQRP ={:.2f}'.format(m1_IQRPT), transform=axes[0][0].transAxes, fontsize = 16)
    axes[0][0].text(0.05, 0.85, 'R2 ={:.2f}'.format(np.round(score1_cv,3)), transform=axes[0][0].transAxes, fontsize = 16)
    axes[0][0].text(0.95, 0.15, 'Method: '+method1, transform=axes[0][0].transAxes, 
                    horizontalalignment='right', fontsize = 20)
    axes[0][0].text(0.95, 0.10, 'smoothing window={:.2f}'.format(m1_wT), transform=axes[0][0].transAxes, 
                    horizontalalignment='right', fontsize = 12)
    if method1 == 'PLSR':
        axes[0][0].text(0.95, 0.05, 'n_component={:.2f}'.format(m1_n_compT), transform=axes[0][0].transAxes, 
                    horizontalalignment='right', fontsize = 12)
    
    
    Ny1_tp.plot.scatter(ax= axes[0][1], x="actual", y="predic", alpha=0.8, color = clr[i], edgecolors='k')
    axes[0][1].plot(Ny1_test, np.polyval(z1N, Ny1_test),  c='blue', linewidth=1)
    axes[0][1].plot(Ny1_test, Ny1_test, color='green', linewidth=1)
    axes[0][1].tick_params(axis='both', labelsize=10)
    axes[0][1].text(0.05, 0.95,  target_names[i]+' (Normalised)', transform=axes[0][1].transAxes,fontsize = 20, color = clr[i])
    axes[0][1].text(0.05, 0.90, 'IQRP ={:.2f}'.format(m1_IQRPNT), transform=axes[0][1].transAxes, fontsize = 16)
    axes[0][1].text(0.05, 0.85, 'R2 ={:.2f}'.format(np.round(score1_cvN,3)), transform=axes[0][1].transAxes, fontsize = 16)
    axes[0][1].text(0.95, 0.15, 'Method: '+method1, transform=axes[0][1].transAxes, 
                    horizontalalignment='right', fontsize = 20)
    axes[0][1].text(0.95, 0.10, 'smoothing window={:.2f}'.format(m1_wNT), transform=axes[0][1].transAxes, 
                    horizontalalignment='right', fontsize = 12)
    if method1 == 'PLSR':
        axes[0][1].text(0.95, 0.05, 'n_component={:.2f}'.format(m1_n_compNT), transform=axes[0][1].transAxes, 
                    horizontalalignment='right', fontsize = 12)
        
        
        
    #with plt.style.context(('ggplot')):  ------- PLOT for Model 2--------------------------------------
    y2_tp.plot.scatter(ax= axes[1][0], x="actual", y="predic", alpha=0.8, color = clr[i], edgecolors='k')
    axes[1][0].plot(y2_test, np.polyval(z2, y2_test),  c='blue', linewidth=1)
    axes[1][0].plot(y2_test, y2_test, color='green', linewidth=1)
    axes[1][0].tick_params(axis='both', labelsize=10)
    axes[1][0].text(0.05, 0.95, target_names[i], transform=axes[1][0].transAxes, fontsize = 20, color = clr[i])
    axes[1][0].text(0.05, 0.90, 'IQRP ={:.2f}'.format(m2_IQRPT), transform=axes[1][0].transAxes, fontsize = 16)
    axes[1][0].text(0.05, 0.85, 'R2 ={:.2f}'.format(np.round(score2_cv,3)), transform=axes[1][0].transAxes, fontsize = 16)
    axes[1][0].text(0.95, 0.15, 'Method: '+method2, transform=axes[1][0].transAxes, 
                    horizontalalignment='right', fontsize = 20)
    axes[1][0].text(0.95, 0.10, 'smoothing window={:.2f}'.format(m2_wT), transform=axes[1][0].transAxes, 
                    horizontalalignment='right', fontsize = 12)
    if method2 == 'PLSR':
        axes[1][0].text(0.95, 0.05, 'n_component={:.2f}'.format(m2_n_compT), transform=axes[1][0].transAxes, 
                    horizontalalignment='right', fontsize = 12)
    
    
    Ny2_tp.plot.scatter(ax= axes[1][1], x="actual", y="predic", alpha=0.8, color = clr[i], edgecolors='k')
    axes[1][1].plot(Ny2_test, np.polyval(z2N, Ny2_test),  c='blue', linewidth=1)
    axes[1][1].plot(Ny2_test, Ny2_test, color='green', linewidth=1)
    axes[1][1].tick_params(axis='both', labelsize=10)
    axes[1][1].text(0.05, 0.95,  target_names[i]+' (Normalised)', transform=axes[1][1].transAxes,fontsize = 20, color = clr[i])
    axes[1][1].text(0.05, 0.90, 'IQRP ={:.2f}'.format(m2_IQRPNT), transform=axes[1][1].transAxes, fontsize = 16)
    axes[1][1].text(0.05, 0.85, 'R2 ={:.2f}'.format(np.round(score2_cvN,3)), transform=axes[1][1].transAxes, fontsize = 16)
    axes[1][1].text(0.95, 0.15, 'Method: '+method2, transform=axes[1][1].transAxes, 
                    horizontalalignment='right', fontsize = 20)
    axes[1][1].text(0.95, 0.10, 'smoothing window={:.2f}'.format(m2_wNT), transform=axes[1][1].transAxes, 
                    horizontalalignment='right', fontsize = 12)
    if method2 == 'PLSR':
        axes[1][1].text(0.95, 0.05, 'n_component={:.2f}'.format(m2_n_compNT), transform=axes[1][1].transAxes, 
                    horizontalalignment='right', fontsize = 12)        

# ax.text(0.95, 0.01, 'colored text in axes coords',
#         verticalalignment='bottom', horizontalalignment='right',
#         transform=ax.transAxes,
#         color='green', fontsize=15)    

#    fig.suptitle('Predictions on "Test" Data', x = 0.5 ,y = .95, fontsize=20)
#     plt.xlim([0, 12])
#     plt.ylim([0, 12])
    #plt.show()

ipywidgets.interact(plot_model_acc, target = target_names, method1 = ['PLSR','MULT'], method2 = ['PLSR','MULT'])

interactive(children=(Dropdown(description='target', options=('sand', 'silt', 'clay', 'TOC'), value='sand'), D…

<function __main__.plot_model_acc(target, method1, method2)>

In [ ]:
spectra = udf.iloc[:, 7:2158].copy()
w1, n1, IQRP1 = best_param_PLSR (spectra, y_sand, rand_sand, 0.2, 3,20)

In [ ]:
spectra = udf.iloc[:, 7:2158].copy()
w2, n2, IQRP2 = best_param_PLSR (spectra, y_silt, rand_silt, 0.2, 3,20)  

In [ ]:
spectra = udf.iloc[:, 7:2158].copy()
w3, n3, IQRP3 = best_param_PLSR (spectra, y_clay, rand_clay, 0.2, 3,20)

In [ ]:
spectra = udf.iloc[:, 7:2158].copy()
w4, n4, IQRP4 = best_param_PLSR (spectra, y_toc, rand_toc, 0.2, 3, 20)

# Step 4: Training the Model

        - Create a linear regr object
        - Apply fit function (using regr object) on training data
        - Check the weights/parameters generated by fit function 

In [ ]:
# spectra = udf.iloc[:,7:2158].copy()
# smth_spec = sgsmooth (spectra,50)

# smth_spec.iloc[1,:].plot()
# plt.show()
# spectra.iloc[1,:].plot()
# plt.show()

In [ ]:
from sklearn import linear_model

regSand = PLSRegression(n_components=n1, scale=True)
regSilt = PLSRegression(n_components=n2, scale=True)
regClay = PLSRegression(n_components=n3, scale=True)
regTOC = PLSRegression(n_components=n4, scale=True)


# train the model using X_train_smth and y_train (identified in previous step)

XSand_train_smth = sgsmooth (XSand_train, w1)       
regSand.fit(XSand_train_smth, ySand_train)

XSilt_train_smth = sgsmooth (XSilt_train, w2)
regSilt.fit(XSilt_train_smth, ySilt_train)

XClay_train_smth = sgsmooth (XClay_train, w3)
regClay.fit(XClay_train_smth, yClay_train)

XTOC_train_smth = sgsmooth (XTOC_train, w4)
regTOC.fit(XTOC_train_smth, yTOC_train)



# Step 5: Prediction on Test Data 

In [ ]:
# Predition using the trained-Model on the "Test Data"

ySand_pred = regSand.predict(XSand_test,copy=True)
ySand_pred = np.round(ySand_pred, 2)

ySilt_pred = regSilt.predict(XSilt_test,copy=True)
ySilt_pred = np.round(ySilt_pred, 2)

yClay_pred = regClay.predict(XClay_test,copy=True)
yClay_pred = np.round(yClay_pred, 2)

yTOC_pred = regTOC.predict(XTOC_test,copy=True)
yTOC_pred = np.round(yTOC_pred, 2)


# Step 6: Model Accuracy and Plots

In [ ]:
# Merging actual and predicted (Test data) in one data frame for plots
ySand_tp = pd.DataFrame({'actual':ySand_test.values, 'predic': ySand_pred[:,0]})
# ySand_sortedA = ySand_tp.sort_values(by =['actual'])
# ySand_srtA = ySand_sortedA.reset_index(drop=True)

ySilt_tp = pd.DataFrame({'actual':ySilt_test.values, 'predic': ySilt_pred[:,0]})
# ySilt_sortedA = ySilt_tp.sort_values(by =['actual'])
# ySilt_srtA = ySilt_sortedA.reset_index(drop=True)

yClay_tp = pd.DataFrame({'actual':yClay_test.values, 'predic': yClay_pred[:,0]})
# yClay_sortedA = yClay_tp.sort_values(by =['actual'])
# yClay_srtA = yClay_sortedA.reset_index(drop=True)

yTOC_tp = pd.DataFrame({'actual':yTOC_test.values, 'predic': yTOC_pred[:,0]})
# yTOC_sortedA = yTOC_tp.sort_values(by =['actual'])
# yTOC_srtA = yTOC_sortedA.reset_index(drop=True)

In [ ]:
# # Plots for Visualising the Model Accuracy
#plt.style.use(['science','notebook','grid'])

def plot_pred_acc (target):

    fig, axes = plt.subplots(1,1, figsize=(8,7))

    if target == 'sand':
        z = np.polyfit(ySand_test, ySand_pred, 1)
        score_cv = r2_score(ySand_test, ySand_pred)
        with plt.style.context(('ggplot')):
            ySand_tp.plot.scatter(ax= axes, x="actual", y="predic", alpha=0.8, color = balu, edgecolors='k')
            axes.plot(np.polyval(z, ySand_test), ySand_test, c='blue', linewidth=1)
            axes.plot(ySand_test, ySand_test, color='green', linewidth=1)
            axes.tick_params(axis='both', labelsize=10)
            plt.text(1, 11, 'SAND', fontsize = 20, color = balu)
            plt.text(1, 10.2, 'IQRP ={:.2f}'.format(IQRP1), fontsize = 16)
            plt.text(1, 9.4, 'R2 ={:.2f}'.format(np.round(score_cv,3)), fontsize = 16)

    elif target == 'clay':
        z = np.polyfit(yClay_test, yClay_pred, 1)
        score_cv = r2_score(yClay_test, yClay_pred)
        with plt.style.context(('ggplot')):
            yClay_tp.plot.scatter(ax= axes, x="actual", y="predic", alpha=0.8, color = mati, edgecolors='k')
            axes.plot(np.polyval(z, yClay_test), yClay_test, c='blue', linewidth=1)
            axes.plot(yClay_test, yClay_test, color='green', linewidth=1)
            axes.tick_params(axis='both', labelsize=10)
            plt.text(1, 11, 'CLAY', fontsize = 20, color = mati)
            plt.text(1, 10.2, 'IQRP ={:.2f}'.format(IQRP3), fontsize = 16)
            plt.text(1, 9.4, 'R2 ={:.2f}'.format(np.round(score_cv,3)), fontsize = 16)

        
    elif target == 'silt':
        z = np.polyfit(ySilt_test, ySilt_pred, 1)
        score_cv = r2_score(ySilt_test, ySilt_pred)
        with plt.style.context(('ggplot')):
            ySilt_tp.plot.scatter(ax= axes, x="actual", y="predic", alpha=0.8, color = kado, edgecolors='k')
            axes.plot(np.polyval(z, ySilt_test), ySilt_test, c='blue', linewidth=1)
            axes.plot(ySilt_test, ySilt_test, color='green', linewidth=1)
            axes.tick_params(axis='both', labelsize=10)
            plt.text(1, 11, 'SILT', fontsize = 20, color = kado)
            plt.text(1, 10.2, 'IQRP ={:.2f}'.format(IQRP2), fontsize = 16)
            plt.text(1, 9.4, 'R2 ={:.2f}'.format(np.round(score_cv,3)), fontsize = 16)


    elif target == 'TOC':
        z = np.polyfit(yTOC_test, yTOC_pred, 1)
        score_cv = r2_score(yTOC_test, yTOC_pred)
        with plt.style.context(('ggplot')):
            yTOC_tp.plot.scatter(ax= axes, x="actual", y="predic", alpha=0.8, color = 'green', edgecolors='k' )
            axes.plot(np.polyval(z, yTOC_test), yTOC_test, c='blue', linewidth=1)
            axes.plot(yTOC_test, yTOC_test, color='green', linewidth=1)
            axes.tick_params(axis='both', labelsize=10)
            plt.text(1, 11, 'TOC', fontsize = 20, color = 'green')
            plt.text(1, 10.2, 'IQRP ={:.2f}'.format(IQRP4), fontsize = 16)
            plt.text(1, 9.4, 'R2 ={:.2f}'.format(np.round(score_cv,3)), fontsize = 16)

        
    else:  
        ySand_srtA.plot.scatter(ax= axes, x="actual", y="predic", alpha=0.8, color = balu)
        axes.tick_params(axis='both', labelsize=10)
        plt.text(3, 11, 'SAND', fontsize = 20, color = balu)
        yClay_srtA.plot.scatter(ax= axes, x="actual", y="predic", alpha=0.8, color = mati)
        axes.tick_params(axis='both', labelsize=10)
        plt.text(3, 10, 'CLAY', fontsize = 20, color = mati)
        ySilt_srtA.plot.scatter(ax= axes, x="actual", y="predic", alpha=0.8, color = kado)
        axes.tick_params(axis='both', labelsize=10)
        plt.text(1, 11, 'SILT', fontsize = 20, color = kado)
        yTOC_srtA.plot.scatter(ax= axes, x="actual", y="predic", alpha=0.8, color = 'green')
        axes.tick_params(axis='both', labelsize=10)
        plt.text(1, 10, 'TOC', fontsize = 20, color = 'green')
     

    fig.suptitle('Predictions on "Test" Data', x = 0.5 ,y = .95, fontsize=20)
    plt.xlim([0, 12])
    plt.ylim([0, 12])
    plt.show()

ipywidgets.interact(plot_pred_acc, target = ['sand', 'clay', 'silt', 'TOC', 'All'])

